<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import numpy as np 
import pandas as pd
import os, io
import datetime as dt
import pandas_datareader.data as web
import requests
from alpha_vantage.timeseries import TimeSeries

In [88]:
def calc_indicators(df):
  df['ema_25'] = calc_ema_25(df)
  df['ema_100'] = calc_ema_100(df)

  df['sma_25'] = calc_sma_25(df)
  df['sma_100'] = calc_sma_100(df)

  calc_bollinger(df)

  calc_macd_signalline(df)

  df['RSI'] = calc_rsi(df)

  df['CCI'] = calc_cci(df)

  # df['OBV_MA'] = calc_obv(df)
   
  df = calculate_dead_cat_bounce_v2(df)

  return df

In [89]:
def calc_ema_25(df):
    ema_25 = 25
    df_ema_25 = df['Adj Close'].ewm(span=ema_25, adjust=False).mean()

    return df_ema_25

def calc_ema_100(df):
    ema_100 = 100
    df_ema_100 = df['Adj Close'].ewm(span=ema_100, adjust=False).mean()

    return df_ema_100

In [90]:
def calc_sma_25(df):
    sma_25 = 25
    df_sma_25 = df['Adj Close'].rolling(window=sma_25).mean()

    return df_sma_25

def calc_sma_100(df):
    sma_100 = 100
    df_sma_100 = df['Adj Close'].rolling(window=sma_100).mean()

    return df_sma_100

In [91]:
def calc_bollinger(df):
    bollinger_avg = 20

    short_avg_ma = df['Adj Close'].rolling(window=bollinger_avg).mean()
    short_avg_std = df['Adj Close'].rolling(window=bollinger_avg).std()

    df['bollinger'] = df['Adj Close'].ewm(span=bollinger_avg, adjust=False).mean()
    df['bollinger_lower_band'] = short_avg_ma - (short_avg_std *2)
    df['bollinger_upper_band'] = short_avg_ma + (short_avg_std *2)

In [92]:
def calc_macd_signalline(df):
    ema_short = 12
    ema_long = 26
    signalline_span = 9

    df_ema_short = df['Adj Close'].ewm(span=ema_short, adjust=False).mean()
    df_ema_long = df['Adj Close'].ewm(span=ema_long, adjust=False).mean()
    df['macd'] = df_ema_short - df_ema_long
    df['signalline'] = df['macd'].ewm(span=signalline_span, adjust=False).mean()

In [93]:
def calc_rsi(df, n=14):
    delta = df['Adj Close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    # Calculate the EWMA
    RolUp = dUp.ewm(span=n).mean()
    RolDown = dDown.abs().ewm(span=n).mean()

    # Calculate the RSI based on EWMA
    RS = RolUp / RolDown
    RSI = 100 - (100/(1+RS))

    # Calculate the SMA
    roll_up2 = dUp.rolling(n).mean()
    roll_down2 = dDown.abs().rolling(n).mean()

    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    own_RSI=(RSI+RSI2)/2
    return own_RSI

In [94]:
def calc_cci(df, n=14, constant=0.015):
    tippingPrice = (df['High'] + df['Low'] + df['Adj Close']) / 3
    tippingPrice_mean = tippingPrice.rolling(n).mean()
    tippingPrice_std = tippingPrice.rolling(n).std()
    
    CCI = pd.Series((tippingPrice - tippingPrice_mean) / (constant * tippingPrice_std)) 
    return CCI

In [95]:
### To Be Adjusted

def calc_obv(df, n = 20):
    i = 0
    OBV = [0]

    while i < df.index[-1]:
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] > 0:
            OBV.append(df.loc[i + 1, 'Volume'])
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] == 0:
            OBV.append(0)
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] < 0:
            OBV.append(-df.loc[i + 1, 'Volume'])
        i = i + 1
    OBV = pd.Series(OBV)
    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean())

    return OBV_ma

In [96]:
def calculate_dead_cat_bounce_v2(df):
  set_freq = '2D'

  df['body_mid'] = (df['Adj Close'] + df['Open'])/2

  pct_change_close = df['body_mid'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((df.signalline > df.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            ((df.Open < df.bollinger_lower_band)|(df.Close < df.bollinger_lower_band)),
                            True, False)

  df['dead_cat_bounce'] = dead_cat_bounce
  del df['body_mid']

  return df

In [97]:
def combine_stocks_to_dataframe(id_list):

  ts = TimeSeries(key='AXUYKPP8SFQ1MAB7', output_format='pandas')
  ts_2 = TimeSeries(key='RO2DTNSBK7N7IVKS', output_format='pandas')
  
  start_date = '2020-7-28'
  df_list = []

  for id in id_list:
    data, meta_data = ts.get_daily_adjusted(symbol=id, outputsize='full') # Change outputsize to 'compact' and only receive data from the last 100 days
    data = data.sort_index(ascending=True)
    data['stock_id'] = id
    
    del data['6. volume']
    del data['7. dividend amount']
    del data['8. split coefficient']

    # Resize df and rename columns appropriately
    stock = data[start_date:].copy()
    stock.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'stock_id']

    # calc indicators for every stock
    stock = calc_indicators(stock) 

    # drop unnecessary rows with NaN values
    stock.dropna(inplace=True) 
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

In [98]:
stocks_dax = ['ADS.DE', 'ALV.DE', 'BAS.DE', 'BAYN.DE', 'BMW.DE', 'CON.DE', '1COV.DE', 'DAI.DE', 'DHER.DE', 'DBK.DE', 'DB1.DE', 'DPW.DE', 'DTE.DE', 'DWNI.DE', 'EOAN.DE', 'FRE.DE', 'FMS', 'HEI.DE', 'HEN3.DE', 'IFX.DE', 'LIN.DE', 'MRK', 'MTX.DE', 'MUV2.DE', 'RWE.DE', 'SAP.DE', 'SIE.DE', 'ENR.F', 'VOW.DE', 'VNA.DE']
stocks_random = ['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE', 'PLUG', 'MDO.DE', 'PYPL', 'V', 'YNDX']

stocks_5_api_calls = ['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE']

stocks_combine = np.concatenate([stocks_dax, stocks_random])
df_complete = combine_stocks_to_dataframe(stocks_5_api_calls)

In [99]:
df_yd = df_complete.drop_duplicates(subset='stock_id', keep='last')

In [100]:
def ema_score(row):
    ema_25 = row['ema_25']
    ema_100 = row['ema_100']
    score = (ema_25/ema_100)-1
    return score

def macd_signalline_score(row): 
    # relative change might be distorted if values near by 0
    macd = row['macd']
    signalline = row['signalline']
    adj_close = row['Adj Close']
    score = (macd/adj_close+1)-(signalline/adj_close+1)
    return score

# set weighting when close is lower lower_band or upper upper_band
def bollinger_score(row):
    lower_band = row['bollinger_lower_band']
    upper_band = row['bollinger_upper_band']
    middle_band = row['bollinger']
    adj_close = row['Adj Close']
    if (adj_close>=middle_band):
        if adj_close>=upper_band:
            score = ((adj_close/upper_band)-1)*(-1)
        else:
            score = ((adj_close/((upper_band-middle_band)*0.5+middle_band))-1)*(-1)
    else:
        if (adj_close<=lower_band):
            score = ((adj_close/lower_band)-1)*(-1)
        else:
            score = (adj_close/middle_band)-1
    return score

def sma_score(row):
    sma_25 = row['sma_25']
    adj_close = row['Adj Close']
    score = (adj_close/sma_25)-1
    return score

def rsi_score(row):
    lower_band = 30
    upper_band = 70
    middle_band = (lower_band + upper_band) / 2 
    middle_band1 = 55
    middle_band2 = 45
    outlier_factor = 1.2
    rsi = row['RSI']
    if (rsi>middle_band):
        if rsi>=upper_band:
            score = ((rsi/upper_band)-1)*(-1)*(outlier_factor)
        else:
            if rsi<middle_band1:
                score = 0
            else:
                if rsi<=62.5:
                    score = rsi/middle_band1-1
                else:
                    score = ((62.5-(rsi-62.5))/(middle_band1))-1
    else:
        if (rsi<=lower_band):
            score = ((rsi/lower_band)-1)*(-1)*(outlier_factor)
        else:
            if rsi>middle_band2:
                score = 0
            else:
                if rsi>=37.5:
                    score = (rsi/middle_band2-1)
                else:
                    score = (37.5-(rsi-37.5))/middle_band2-1
    return score

# We should check whether we give the same weighting if it hits above or lower 100/-100
def cci_score(row):
    lower_band = -100
    middle_lower_band = -50
    upper_band = 105
    middle_band = 0
    middle_upper_band = 85
    outlier_factor = 0.25
    outlier_factor1 = 2
    cci=row['RSI']

    if (cci < lower_band):
        score=(cci/lower_band)-1
    else:
        if (cci<=middle_lower_band):
            score = (cci / lower_band)-1
        elif (cci<middle_band):
            score = (((cci-lower_band)/lower_band)+1)*(-1)
        else:
            if (cci>upper_band):
                score = ((cci/upper_band)-1)*(-1)
            else:
                if(cci<=middle_upper_band):
                    score = (cci/upper_band)*outlier_factor
                else:
                    if(cci<upper_band):
                        score=(middle_upper_band-(cci-middle_upper_band))/upper_band-(middle_upper_band-(middle_upper_band-upper_band))/middle_upper_band
                    else:
                        score=0
    
    return score


    ## TO BE ADDED: DEAD-CAT-BOUNCE SCORE CALC

In [101]:
def relative_score(row):
    r = row
    stock = row['stock_id]

    
    ema_scr = ema_score(r)
    macd_signalline_scr = macd_signalline_score(r)
    bollinger_scr = bollinger_score(r)
    sma_scr = sma_score(r)
    rsi_scr = rsi_score(r)
    cci_scr = cci_score(r)

    score = ema_scr + (macd_signalline_scr*5) + bollinger_scr + sma_scr + rsi_scr + cci_scr
    print(stock, ema_scr, (macd_signalline_scr*5), bollinger_scr, sma_scr, rsi_scr, cci_scr, score, '\n')
    return score

In [102]:
df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)
df_yd.sort_values(by=['score'], ascending=False)

AMZN 0.053671719058725476 -0.015398326031249576 0.01035264357255361 0.025016106979496966 0.1222524338433355 0.15065741937765845 0.3465519968005204 

AAPL 0.06686337533614584 -0.0032891182518623996 -0.006068647444546826 0.05489786180099454 0.07112801163359661 0.15735228419083852 0.3408837672651663 

SAP 0.03676758971059768 -0.040109936926839285 -0.03232013879297624 -0.03176875760518871 -0.13096896982848927 0.08546096105305243 -0.1129392523898434 

ACN 0.06725944808749973 0.002797902569963151 -0.010990472442474752 0.052299820462732516 -0.1099326359131509 0.18193508832127095 0.18336915108584068 

VOW.DE 0.056431524877070816 0.008336245622158622 0.008435776605846046 0.011341675279371355 0 0.12652117603615037 0.2110663984205972 

<ipython-input-102-8c16aa146636>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,Open,High,Low,Close,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce,score
date,,,,,,,,,,,,,,,,,,,
2021-07-23,3640.00,3666.1100,3622.04,3656.64,3656.64,AMZN,3559.536533,3378.221574,3567.397600,3317.925100,3580.920988,3369.711190,3808.862810,56.018622,67.279849,63.276116,1.476010,False,0.346552
2021-07-23,147.55,148.7177,146.92,148.56,148.56,AAPL,141.451142,132.585995,140.828800,130.188055,142.796787,133.156027,152.530973,3.940260,4.037986,66.087959,77.051849,False,0.340884
2021-07-23,277.80,283.2000,276.20,279.20,279.20,VOW.DE,276.930368,262.137546,276.068916,274.889209,276.519278,262.779666,286.631334,-1.147030,-1.612526,53.138894,15.218117,False,0.211066
2021-07-23,316.00,318.8000,315.54,318.65,318.65,ACN,304.876734,285.663186,302.812938,284.673663,307.167815,291.254319,323.204100,6.918266,6.739956,76.412737,113.737083,False,0.183369
2021-07-23,140.01,140.0600,138.04,138.77,138.77,SAP,143.231018,138.151520,143.323200,135.693865,143.404865,136.251219,151.842781,-0.007828,1.105383,35.893604,-101.618253,False,-0.112939


In [103]:
def bot_warn_me(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '1790740607:AAE7Omfuea3tG3iii9A1qX-vQJ9pdLFcO0s'
    bot_chat_id = '908941210'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()